_Autor:_    __Jesús Casado__ <br> _Revisión:_ __27/08/2019__ <br>


__Descripción__:<br>
Se unen las series del SAIH Cantábrico obtenidas de tres peticiones distintas: junio de 2015, junio de 2018 y junio de 2019.

Se unen las series para que tengan un formato único y se agregan a resolución diaria (resolución original cincominutal).

Se crea una tabla con las estaciones con datos, las variables que se miden en cada una de ellas, y el inicio y fin de la serie.

Se comprueba la disponibilidad de datos en cada periodo.

__Cosas a corregir__ <br>


__Índice__<br>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-dark')
%matplotlib inline
from simpledbf import Dbf5
import os
from datetime import datetime, timedelta
from shutil import copyfile

In [2]:
%run funciones_seriesCHC.ipynb

### Configuración
#### Cargar datos

In [3]:
rutaSAIH = 'F:/OneDrive - Universidad de Cantabria/Series/CHC/SAI/'

In [4]:
# Cargar datos de las estaciones
stn_coord = pd.read_excel(rutaSAIH + 'GIS/Coordenadas estaciones_CHC-IHCantabria_2019.xls')
stns = stn_coord['COD_SAIH'].tolist()
stns = stns + ['A607', 'A060']
stns.sort()

print('nº de estaciones:', len(stns))

nº de estaciones: 84


#### Reordenar archivos

In [5]:
move = False

if move == True:
    # mover archivos a la carpeta adecuada
    rutaOrig = rutaSAIH + 'originales/2018.06-2019.05/'
    rutaCopy = rutaSAIH + 'originales/Desde julio de 2015/'

    # seleccionar mes
    for mes in os.listdir(rutaOrig):
        ruta1 = rutaOrig + mes + '/Cincominutales/'
        file = os.listdir(ruta1)
        # encontrar archivos csv
        for file in os.listdir(ruta1):
            stn = file[8:12]
            ruta2 = rutaCopy + stn + '/'
            if not os.path.exists(ruta2):
                os.makedirs(ruta2)
            copyfile(ruta1 + file, ruta2 + file)

### Unir series

In [6]:
# ruta de las series ya tratadas
rutaOrig = rutaSAIH + 'originales/'
# ruta de las nuevas series originales
rutaExp = rutaSAIH + 'tratadas/'

In [11]:
Input = newstns#['A624', 611', 'A617', 'Q106']
for i, stn in enumerate(Input):
    print('estación ', str(stn), '\t(', i+1, ' de ', len(Input), ')',
          sep='')
    SAIH_CHC(stn, rutaOrig, freq='1D', export_orig=True, rutaexp=rutaExp)
    print()

estación A080	(1 de 1)
nº de días de las serie:	 3370
variables:	 ['caudal_m³/s', 'nivel_m']



### Estaciones y variables disponibles

In [ ]:
# Variables disponibles y periodo de obervación en cada estación
# --------------------------------------------------------------

# crear la tabla
variables = ['precipitacion', 'caudal', 'nivel', 'temperatura',
             'amonio', 'conductividad', 'oxigeno', 'pH',
             'temperaturaAgua', 'turbidez']# , 'piezometro', 'limnimetro'
res = pd.DataFrame(index=stns, columns=variables + ['inicio', 'fin'])
res.index.name = 'COD_SAIH'
res.loc[:,:] = 0

# completar la tabla
ruta = rutaExp + '5min/'
for stn in stns:
    aux = pd.read_csv(ruta + stn + '.csv', encoding='latin1', parse_dates=True,
                      index_col=0)
    for col in aux.columns:
        res.loc[stn, col.split('_')[0]] = 1
    res.loc[stn, 'inicio'] = datetime.strftime(aux.index[0], format='%d-%m-%Y')
    res.loc[stn, 'fin'] = datetime.strftime(aux.index[-1], format='%d-%m-%Y')

res.columns = ['precip', 'caudal', 'nivel', 'Taire', 'amonio', 'conduct',
               'oxigeno', 'pH', 'Tagua', 'turbidez', 'inicio', 'fin']
               #, 'piezo', 'limni'

# exportar
res.to_csv(rutaSAIH + 'GIS/estaciones y variables.csv', header=True)

res.head()

In [ ]:
res.sum()

### Comparación de datos en los 3 periodos
#### En cada periodo por separado

In [ ]:
Orig = pd.DataFrame()#index=stns, columns=cols)

# Estaciones disponibles en los periodos 1 y 2 separadas en Cantabria y fuera
for i, loc in enumerate(['Cantabria', 'noCantabria']):
    for j, time in enumerate(['Hasta junio de 2015', 'Desde julio de 2015']):
        stn_aux = [folder[:4] for folder in os.listdir(rutaOrig + loc + '/' + time + '/')]
        aux = pd.Series(data=1, index=stn_aux)
        Orig = pd.concat((Orig, aux), axis=1)
        del aux, stn_aux
Orig.columns = ['Cant1', 'Cant2', 'noCant1', 'noCant2']

# Estaciones disponibles en los periodos 1 y 2 en toda el área
Orig['todo1'] = np.nan
Orig['todo2'] = np.nan
for stn in Orig.index:
    if (Orig.loc[stn, 'Cant1'] == 1) | (Orig.loc[stn, 'noCant1'] == 1):
        Orig.loc[stn, 'todo1'] = 1
        if (Orig.loc[stn, 'Cant1'] == 1) & (Orig.loc[stn, 'noCant1'] == 1):
            print(stn)
    if (Orig.loc[stn, 'Cant2'] == 1) | (Orig.loc[stn, 'noCant2'] == 1):
        Orig.loc[stn, 'todo2'] = 1

# Estaciones disponibles en el periodo 3 en toda el área
ruta = rutaOrig + '2018.06-2019.05/'
stn_aux = []
# seleccionar mes
for mes in os.listdir(ruta):
    ruta1 = ruta + mes + '/Cincominutales/'
    file = os.listdir(ruta1)
    # encontrar archivos csv
    for file in os.listdir(ruta1):
        stn_aux.append(file[8:12])
        stn_aux = list(set(stn_aux))
stn_aux.sort()
aux = pd.Series(data=1, index=stn_aux)
Orig = pd.concat((Orig, aux), axis=1)

Orig.columns = list(Orig.columns)[:-1] + ['todo3']

Orig.shape

In [ ]:
Orig.sum()

Las estaciones totales disponibles en cada periodo no coinciden. 
*  El primer periodo cuenta con 2 estaciones que no aparecen en los dos posteriores; según la fecha de fin, dichas estaciones dejaron de funcionar.
*  En el periodo 2 aparecen 7 nuevas estaciones que se mantienen en el periodo 3.
*  En el periodo 3 aperecen 3 nuevas estaciones. Tengo dudas de que la estación A602 no existiera antes del 31 de mayo de 2018.

##### Estaciones disponibles en cada uno de los tres periodos

In [ ]:
stns1 = Orig[Orig['todo1'] == 1].index
stns2 = Orig[Orig['todo2'] == 1].index
stns3 = Orig[Orig['todo3'] == 1].index

In [ ]:
fallo1 = set(stns1) - set(stns2)
print('estaciones disponibles en el 1º periodo y no en el 2º:', len(fallo))
print(fallo1)
print()
fallo2 = set(stns1) - set(stns3)
print('estaciones disponibles en el 1º periodo y no en el 3º:', len(fallo))
print(fallo2)

res.loc[fallo2, ['inicio', 'fin']]

In [ ]:
fallo1 = set(stns2) - set(stns1)
print('estaciones disponibles en el 2º periodo y no en el 1º:', len(fallo))
print(fallo1)
print()
fallo2 = set(stns2) - set(stns2)
print('estaciones disponibles en el 2º periodo y no en el 3º:', len(fallo))
print(fallo2)

res.loc[fallo1, ['inicio', 'fin']]

In [ ]:
fallo1 = set(stns3) - set(stns1)
print('estaciones disponibles en el 3er periodo y no en el 1º:', len(fallo))
print(fallo1)
print()
fallo2 = set(stns3) - set(stns2)
print('estaciones disponibles en el 3er periodo y no en el 2º:', len(fallo))
print(fallo2)

res.loc[fallo2, ['inicio', 'fin']]

#### Periodos combinados

In [ ]:
Orig2 = pd.DataFrame()#index=stns, columns=cols)

for j, time in enumerate(['Hasta junio de 2015', 'Desde julio de 2015']):
    stn_aux = [folder[:4] for folder in os.listdir(rutaOrig + time + '/')]
    aux = pd.Series(data=1, index=stn_aux)
    Orig2 = pd.concat((Orig2, aux), axis=1)
    del aux
Orig2.columns = ['todo1', 'todo2+3']

Orig2.shape

In [ ]:
Orig2.sum()

El número de estaciones en _todo1_ y _todo3_ coincide en ambas carpetas, por lo que no hay errores en la reorganización de los datos.